In [ ]:
from scipy.integrate import ode, solve_ivp
import os, sys, time
import numpy as np
import matplotlib.pyplot as plt
import pickle
import bisect

import multiprocessing
from functools import partial 

from ds_generator_myokit import Generator
sys.path.append('../')
import plot_cell

sys.path.append('./Protocols')
from protocols import optimized_VC_Christini

In [ ]:
def main():
    
    data_type = 'VC'  # AP or VC    
    
    multi_processing = False
    nData_per_file = 10000
    startNo = 1
    nDataset = 50
    datasetNo_li = list(range(startNo, startNo+nDataset))  # Core 수만큼  [1,2,3,4,5,6,7,8,9,10]      
    print(datasetNo_li)          
    
    cell_types = ['Endocardial', 'Epicardial', 'Midmyocardial']
    sim_params = {
        'cell.mode': 1,         # 'Endocardial' : 0,    'Epicardial' : 1,    'Mid-myocardial' : 2,     
    }
    
    dataset_dir = "./Dataset-%s-%s"%(cell_types[sim_params['cell.mode']], 'ChristiniVC')
    if not os.path.exists(dataset_dir):
        os.makedirs(dataset_dir)
            
    generator = None
    times = None
    log_li = []
    if data_type == 'AP':
        model_path = "./mmt-model-files/newordherg_qNet.mmt" 
        generator = Generator(model_path, pre_sim=1)     
        generator.set_simulation_params(sim_params)
        times = np.arange(0, 1000, 1)# np.linspace(0, simulator.total_duration, 20000)          
    
    elif data_type == 'VC':
        model_path = "./mmt-model-files/newordherg_qNet_VC.mmt" 
        generator = Generator(model_path, optimized_VC_Christini, pre_sim=2)     
        generator.set_simulation_params(sim_params)
        times = np.arange(0, generator.protocol_total_duration, 1)# np.linspace(0, simulator.total_duration, 20000)  
        log_li = ['ina.INa', 'inal.INaL', 'ito.Ito', 'ical.ICaL', 'ikr.IKr', 'iks.IKs', 'ik1.IK1', 'if.iF']
              
    gen_params = {
        'times': times,                    
        'log_li' :  log_li,           
        'nData' : nData_per_file,    
        'dataset_dir' : dataset_dir,        
        'data_file_name' : 'current',
        'scale' : 2,
    }         
        
    start_time = time.time()
    if multi_processing == True:
        # Use multiprocessing
        nProcess = nDataset
        if nDataset>72:
            nProcess = 35
        pool = multiprocessing.Pool(processes=nProcess)
        func = partial(generator.gen_dataset, gen_params)
        pool.map(func, datasetNo_li)
        pool.close()
        pool.join()    
        
    else:
        # Use single processing
        for datasetNo in datasetNo_li:
            generator.gen_dataset( gen_params, datasetNo)
   
    print("All Datasets have been generated.")
    
    print("--- %s seconds ---"%(time.time()-start_time))

In [ ]:
if __name__=='__main__':
    main()

[1, 2]
./Protocols/trial_steps_ramps_Kernik_200_50_4_-120_60/shortened_trial_steps_ramps_Kernik_200_50_4_-120_60_500_artefact_True_short.pkl
The protocol is 9055.950603217367 ms
-----Dataset1 generation starts.-----


100%|██████████| 10/10 [00:01<00:00,  6.64it/s]


=====Dataset1 generation End.===== 0 simulation errors occured.=====
-----Dataset2 generation starts.-----


100%|██████████| 10/10 [00:01<00:00,  7.33it/s]

=====Dataset2 generation End.===== 0 simulation errors occured.=====
All Datasets have been generated.
--- 2.9240522384643555 seconds ---
